In [1]:
from itertools import chain
from pathlib import Path
from typing import Hashable
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Embedding, TimeDistributed, Dense, Dropout, Conv1D, GRU, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from deepod.metrics import tabular_metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import ast
import torch

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
plaid = np.load('../out/plaid.npz', allow_pickle=True)
plaid = plaid['arr_0']
atk = plaid[3]
normal = plaid[0] + plaid[1] + plaid[2]


In [3]:
np.random.shuffle(normal)
train_X = normal[:37000]
test_X = normal[37000:] + atk
max_length = 2000
train_X = pad_sequences(train_X, maxlen=max_length, padding='post')
test_X = pad_sequences(test_X, maxlen=max_length, padding='post')
test_Y = []
norm_test = len(normal) - len(train_X)
for i in range(norm_test):
    test_Y.append(0)
for i in range(len(atk)):
    test_Y.append(1)
print(len(test_X), len(test_Y))

2323 2323


In [4]:
train_X = np.array(train_X)
test_X = np.array(test_X)
test_Y = np.array(test_Y)

In [5]:
from deepod.models.time_series import *

In [6]:
clf = COUTA()
clf.device = torch.device('cpu')
clf.fit(train_X)
scores = clf.predict(test_X)

|>>> epoch: 01  |   loss: 231.847519, loss_oc: 231.764923, val_loss: 14.359663
|>>> epoch: 02  |   loss: 9.787758, loss_oc: 9.726124, val_loss: 10.077424
|>>> epoch: 03  |   loss: 7.542758, loss_oc: 7.488843, val_loss: 8.115606
|>>> epoch: 04  |   loss: 6.369025, loss_oc: 6.318940, val_loss: 6.970551
|>>> epoch: 05  |   loss: 5.511364, loss_oc: 5.463111, val_loss: 5.891752
|>>> epoch: 06  |   loss: 4.935425, loss_oc: 4.889462, val_loss: 5.440456
|>>> epoch: 07  |   loss: 4.487973, loss_oc: 4.443641, val_loss: 5.019156
|>>> epoch: 08  |   loss: 4.387630, loss_oc: 4.343724, val_loss: 4.661003
|>>> epoch: 09  |   loss: 3.950074, loss_oc: 3.907278, val_loss: 4.280840
|>>> epoch: 10  |   loss: 3.782739, loss_oc: 3.741175, val_loss: 4.021345
|>>> epoch: 11  |   loss: 3.661651, loss_oc: 3.620346, val_loss: 3.871691
|>>> epoch: 12  |   loss: 3.468693, loss_oc: 3.427814, val_loss: 3.670423
|>>> epoch: 13  |   loss: 3.299825, loss_oc: 3.259631, val_loss: 3.626865
|>>> epoch: 14  |   loss: 3.2872

In [7]:
precision = precision_score(test_Y, scores, pos_label=0)
recall = recall_score(test_Y, scores, pos_label=0)
f1 = f1_score(test_Y, scores, pos_label=0)
print("Precision: ",precision)
print("Recall: ",recall)
print("f1: ",f1)

Precision:  0.6373559733171619
Recall:  0.8921901528013583
f1:  0.7435443933498408


In [11]:
clf = TimesNet()
clf.device = torch.device('cpu')
clf.fit(train_X)
scores = clf.predict(test_X)

epoch  1, training loss: 885.841881, time: 86.6s
epoch  2, training loss: 591.055130, time: 85.9s
epoch  3, training loss: 489.303961, time: 82.8s
epoch  4, training loss: 589.308538, time: 81.8s
epoch  5, training loss: 611.968646, time: 82.0s
epoch  6, training loss: 522.994518, time: 89.7s
epoch  7, training loss: 427.165651, time: 87.4s
epoch  8, training loss: 463.557782, time: 87.7s
epoch  9, training loss: 442.421362, time: 86.8s
epoch 10, training loss: 511.625387, time: 86.8s


In [12]:
precision = precision_score(test_Y, scores, pos_label=0)
recall = recall_score(test_Y, scores, pos_label=0)
f1 = f1_score(test_Y, scores, pos_label=0)
print("Precision: ",precision)
print("Recall: ",recall)
print("f1: ",f1)

Precision:  0.6113931190073322
Recall:  0.9202037351443124
f1:  0.7346662148424263


In [10]:
clf = DeepSVDDTS()
clf.device = torch.device('cpu')
clf.fit(train_X)
scores = clf.predict(test_X)

Start Training...
ensemble size: 1
TSTransformerEncoder(
  (project_inp): TokenEmbedding(
    (tokenConv): Conv1d(2000, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
  )
  (pos_enc): FixedPositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (conv1): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
        (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p

testing: 100%|██████████████████████████████████| 36/36 [00:13<00:00,  2.57it/s]


In [12]:
precision = precision_score(test_Y, scores, pos_label=0)
recall = recall_score(test_Y, scores, pos_label=0)
f1 = f1_score(test_Y, scores, pos_label=0)
print("Precision: ",precision)
print("Recall: ",recall)
print("f1: ",f1)

Precision:  0.9445958219800181
Recall:  0.8828522920203735
f1:  0.912681000438789


In [7]:
clf = AnomalyTransformer()
clf.device = torch.device('cpu')
clf.fit(train_X)
scores = clf.predict(test_X)

epoch  1, training loss: 872.461337, time: 46.1s
epoch  2, training loss: 857.227894, time: 47.4s
epoch  3, training loss: 820.690885, time: 47.0s
epoch  4, training loss: 830.306050, time: 47.0s
epoch  5, training loss: 796.937078, time: 47.4s
epoch  6, training loss: 799.920102, time: 48.0s
epoch  7, training loss: 783.103460, time: 47.5s
epoch  8, training loss: 776.291598, time: 48.3s
epoch  9, training loss: 788.334292, time: 47.4s
epoch 10, training loss: 779.085544, time: 47.9s


In [8]:
precision = precision_score(test_Y, scores, pos_label=0)
recall = recall_score(test_Y, scores, pos_label=0)
f1 = f1_score(test_Y, scores, pos_label=0)
print("Precision: ",precision)
print("Recall: ",recall)
print("f1: ",f1)

Precision:  0.6376226197345644
Recall:  0.9380305602716469
f1:  0.7591892820336654


In [9]:
clf = DeepIsolationForestTS()
clf.device = torch.device('cpu')
clf.fit(train_X)
scores = clf.predict(test_X)

Start Training...


  0%|                                                    | 0/50 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/deepod/core/networks/network_utility.py:23: UserWarning: use the first hidden num, the rest hidden numbers are deprecated
  warnings.warn('use the first hidden num, '
  2%|▊                                         | 1/50 [01:19<1:04:48, 79.35s/it]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/deepod/core/networks/network_utility.py:23: UserWarning: use the first hidden num, the rest hidden numbers are deprecated
  warnings.warn('use the first hidden num, '
  4%|█▋                                        | 2/50 [02:40<1:04:07, 80.15s/it]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/deepod/core/networks/network_utility.py:23: UserWarning: use the first hidden num, the rest hidden numbers are deprecated
  warnings.warn('use the first hidden num, '
  6%|██▌           

Start Inference on the training data...
Start Inference...


100%|█████████████████████████████████████████| 50/50 [1:07:51<00:00, 81.44s/it]


Start Inference...


100%|███████████████████████████████████████████| 50/50 [03:15<00:00,  3.91s/it]


In [10]:
precision = precision_score(test_Y, scores, pos_label=0)
recall = recall_score(test_Y, scores, pos_label=0)
f1 = f1_score(test_Y, scores, pos_label=0)
print("Precision: ",precision)
print("Recall: ",recall)
print("f1: ",f1)

Precision:  0.9391602399314481
Recall:  0.9303904923599321
f1:  0.9347547974413646


In [11]:
max_num = 0
for i in range(len(normal)):
    for j in range(len(normal[i])):
        max_num = max(max_num, normal[i][j])
print(max_num)
for i in range(len(atk)):
    for j in range(len(atk[i])):
        max_num = max(max_num, atk[i][j])
print(max_num)
# print(max_len)

221.0
228.0


In [6]:
K = 229
val_X = train_X[30000:]
train_X = train_X[:30000]

In [7]:
model = Sequential()
model.add(Embedding(input_dim=K, output_dim=400, input_length=None))
model.add(LSTM(400, return_sequences=True))
model.add(Dropout(0.5))
model.add(Dense(K, activation='softmax'))  

early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the model weights from the epoch with the best value of the monitored metric
)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_crossentropy'])

In [8]:
def add_train_labels(x):
        return x[:-1], x[1:]
    
train = [add_train_labels(seq) for seq in train_X]
val = [add_train_labels(seq) for seq in val_X]

train_inputs, train_targets = zip(*train)
val_inputs, val_targets = zip(*val)

# Convert to numpy arrays
train_inputs = np.array(train_inputs)
train_targets = np.array(train_targets)
val_inputs = np.array(val_inputs)
val_targets = np.array(val_targets)

In [9]:
model.fit(
            train_inputs,
            train_targets,
            validation_data=(val_inputs, val_targets),
            epochs=20,
            verbose=2,
            shuffle=True,
            callbacks=[early_stopping],
        )

Epoch 1/20
938/938 - 4640s - loss: 0.2133 - sparse_categorical_crossentropy: 0.2133 - val_loss: 0.0670 - val_sparse_categorical_crossentropy: 0.0670 - 4640s/epoch - 5s/step
Epoch 2/20
938/938 - 4562s - loss: 0.0469 - sparse_categorical_crossentropy: 0.0469 - val_loss: 0.0273 - val_sparse_categorical_crossentropy: 0.0273 - 4562s/epoch - 5s/step
Epoch 3/20
938/938 - 4657s - loss: 0.0284 - sparse_categorical_crossentropy: 0.0284 - val_loss: 0.0205 - val_sparse_categorical_crossentropy: 0.0205 - 4657s/epoch - 5s/step
Epoch 4/20
938/938 - 4737s - loss: 0.0226 - sparse_categorical_crossentropy: 0.0226 - val_loss: 0.0173 - val_sparse_categorical_crossentropy: 0.0173 - 4737s/epoch - 5s/step
Epoch 5/20


KeyboardInterrupt: 

In [16]:
val = val_X
y_val = model.predict(val)
sums_val = np.array([-np.log(pred.max(axis=-1)).sum(axis=-1) for pred in y_val])
threshold = np.mean(sums_val) + 2*np.std(sums_val)
print(threshold)

219/219 [==============================] - 492s 2s/step
47.29902958869934


In [18]:
test = test_X
print(len(test))
y_pred = model.predict(test)
sums = np.array([-np.log(pred.max(axis=-1)).sum(axis=-1) for pred in y_pred])

2323
73/73 [==============================] - 168s 2s/step


In [21]:
threshold = np.mean(sums_val) + np.std(sums_val)
successes = 0
total = len(test)
pred = []
norms_len = total - len(atk)
testY = []
for i in range(total):
    if sums[i] <= threshold:
        pred.append(0)
    else:
        pred.append(1)
    if i < norms_len:
        testY.append(0)
        if sums[i] <= threshold:
            successes += 1
    else:
        testY.append(1)
        if sums[i] > threshold:
            successes += 1
print(successes)
print(total)
print(successes / total)

1361
2323
0.5858803271631511


In [22]:
precision = precision_score(testY, pred, pos_label=0)
recall = recall_score(testY, pred, pos_label=0)
f1 = f1_score(testY, pred, pos_label=0)
print("Precision: ",precision)
print("Recall: ",recall)
print("f1: ",f1)

Precision:  0.5514775977121068
Recall:  0.9821731748726655
f1:  0.7063492063492064
